In [1]:
cd ~/SageMaker/TemporalFusionTransformer/TFT_docker

/home/ec2-user/SageMaker/TemporalFusionTransformer/TFT_docker


In [2]:
! pwd

/home/ec2-user/SageMaker/TemporalFusionTransformer/TFT_docker


In [3]:
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
! docker build -t pytorch-tft-container-test .

Sending build context to Docker daemon  11.78kB
Step 1/6 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.1-cpu-py36-ubuntu16.04
 ---> 6177bf009b27
Step 2/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 2519ad63144a
Step 3/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> d6a1e08eee2d
Step 4/6 : COPY TFT.py /opt/ml/code/TFT.py
 ---> Using cache
 ---> 28cecfb1fce4
Step 5/6 : RUN pip install pytorch-forecasting
 ---> Running in 04852d3dbed3
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

The command '/bin/sh -c pip install pytorch-forecasting' returned a non-zero code: 1


In [ ]:
!df -h
